# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 11 2022 8:30AM,241457,10,8249216,Eywa Pharma Inc.,Released
1,Jul 11 2022 8:30AM,241460,10,PRF-32969,Bio-PRF,Released
2,Jul 11 2022 8:30AM,241460,10,PRF-32977,Bio-PRF,Released
3,Jul 11 2022 8:30AM,241460,10,PRF-32981,Bio-PRF,Released
4,Jul 11 2022 8:30AM,241460,10,PRF-32984,Bio-PRF,Released
5,Jul 11 2022 8:30AM,241460,10,PRF-32987,Bio-PRF,Released
6,Jul 11 2022 8:30AM,241460,10,PRF-32994,Bio-PRF,Released
7,Jul 11 2022 8:30AM,241460,10,PRF-33000,Bio-PRF,Released
8,Jul 8 2022 3:52PM,241461,19,DEX0007154,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jul 8 2022 3:47PM,241459,20,EXACT-8249217,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
56,241457,Released,1
57,241458,Released,1
58,241459,Released,1
59,241460,Released,7
60,241461,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241457,NaN,NaN,1.0
241458,NaN,NaN,1.0
241459,NaN,NaN,1.0
241460,NaN,NaN,7.0
241461,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241362,0.0,0.0,17.0
241366,24.0,1.0,0.0
241368,0.0,2.0,16.0
241370,0.0,0.0,1.0
241376,1.0,5.0,10.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241362,0,0,17
241366,24,1,0
241368,0,2,16
241370,0,0,1
241376,1,5,10


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241362,0,0,17
1,241366,24,1,0
2,241368,0,2,16
3,241370,0,0,1
4,241376,1,5,10


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241362,,,17
1,241366,24,1,
2,241368,,2,16
3,241370,,,1
4,241376,1,5,10


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF
8,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
9,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc."
10,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd"
11,Jul 8 2022 3:36PM,241456,19,"AdvaGen Pharma, Ltd"
26,Jul 8 2022 3:23PM,241455,102,"Senseonics, Incorporated"
27,Jul 8 2022 3:09PM,241453,102,"Senseonics, Incorporated"
28,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation
56,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,,,1
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,,,7
2,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,1
3,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc.",,,1
4,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd",,,1
5,Jul 8 2022 3:36PM,241456,19,"AdvaGen Pharma, Ltd",,,15
6,Jul 8 2022 3:23PM,241455,102,"Senseonics, Incorporated",,,1
7,Jul 8 2022 3:09PM,241453,102,"Senseonics, Incorporated",,,1
8,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,,,28
9,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,,,46


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,,
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,,
2,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,,
3,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc.",1,,
4,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd",1,,
5,Jul 8 2022 3:36PM,241456,19,"AdvaGen Pharma, Ltd",15,,
6,Jul 8 2022 3:23PM,241455,102,"Senseonics, Incorporated",1,,
7,Jul 8 2022 3:09PM,241453,102,"Senseonics, Incorporated",1,,
8,Jul 8 2022 3:05PM,241450,10,ISDIN Corporation,28,,
9,Jul 8 2022 3:04PM,241449,10,ISDIN Corporation,46,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1,,
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7,,
2,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,,
3,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc.",1,,
4,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7.0,NaN,NaN
2,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,NaN,NaN
3,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc.",1.0,NaN,NaN
4,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 11 2022 8:30AM,241457,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Jul 11 2022 8:30AM,241460,10,Bio-PRF,7.0,0.0,0.0
2,Jul 8 2022 3:52PM,241461,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,0.0,0.0
3,Jul 8 2022 3:47PM,241459,20,"Exact-Rx, Inc.",1.0,0.0,0.0
4,Jul 8 2022 3:46PM,241458,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6518128,258.0,9.0,2.0
102,965756,4.0,0.0,0.0
15,241378,124.0,5.0,39.0
16,965760,7.0,12.0,0.0
18,241422,0.0,1.0,0.0
19,1207179,17.0,2.0,24.0
20,965666,24.0,9.0,3.0
22,724326,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6518128,258.0,9.0,2.0
1,102,965756,4.0,0.0,0.0
2,15,241378,124.0,5.0,39.0
3,16,965760,7.0,12.0,0.0
4,18,241422,0.0,1.0,0.0
5,19,1207179,17.0,2.0,24.0
6,20,965666,24.0,9.0,3.0
7,22,724326,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,258.0,9.0,2.0
1,102,4.0,0.0,0.0
2,15,124.0,5.0,39.0
3,16,7.0,12.0,0.0
4,18,0.0,1.0,0.0
5,19,17.0,2.0,24.0
6,20,24.0,9.0,3.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,258.0
1,102,Released,4.0
2,15,Released,124.0
3,16,Released,7.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,22
Status,,,,,,,,
Completed,2.0,0.0,39.0,0.0,0.0,24.0,3.0,0.0
Executing,9.0,0.0,5.0,12.0,1.0,2.0,9.0,0.0
Released,258.0,4.0,124.0,7.0,0.0,17.0,24.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,22
0,Completed,2.0,0.0,39.0,0.0,0.0,24.0,3.0,0.0
1,Executing,9.0,0.0,5.0,12.0,1.0,2.0,9.0,0.0
2,Released,258.0,4.0,124.0,7.0,0.0,17.0,24.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,22
0,Completed,2.0,0.0,39.0,0.0,0.0,24.0,3.0,0.0
1,Executing,9.0,0.0,5.0,12.0,1.0,2.0,9.0,0.0
2,Released,258.0,4.0,124.0,7.0,0.0,17.0,24.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()